# Solar Array Installation Suitability
#### GIS 5571: Project Rough Draft
#### Author: Jacob Harris
---
For this project, I'm using National Land Cover Data (NLCD) from 2019, MN County Boundary data, and a Digital Elevation Model to perform site suitability analysis for solar panel array installations in Hennepin County, MN. 

First, let's set up our ETL by gathering the necessary data and merging it into a single directory.

In [1]:
import arcpy
import os
import pandas as pd
import random
import requests
import shutil
import zipfile

In [10]:
# Function to download and extract a dataset
def download_and_extract(url, target_path, extract_path):
    response = requests.get(url)
    if response.status_code == 200:
        with open(target_path, 'wb') as file:
            file.write(response.content)
        with zipfile.ZipFile(target_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)

# URLs for the datasets
NLCD_url = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/biota_landcover_nlcd_mn_2019/tif_biota_landcover_nlcd_mn_2019.zip'
DEM_url = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/elev_30m_digital_elevation_model/fgdb_elev_30m_digital_elevation_model.zip'
county_url = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/bdry_counties_in_minnesota/shp_bdry_counties_in_minnesota.zip'

# Directory where you want to save the datasets
base_dir = r"C:\Users\jake1\OneDrive\Documents\ArcGIS\Projects\GIS_5571_Project"
# Create new directories for unzipped data
unzipped_NLCD_dir = os.path.join(base_dir, 'Unzipped Landcover')
unzipped_DEM_dir = os.path.join(base_dir, 'Unzipped Elevation')
unzipped_county_dir = os.path.join(base_dir, 'Unzipped Counties')
# Download and extract the landcover dataset
NLCD_path = os.path.join(base_dir, 'landcover.zip')
download_and_extract(NLCD_url, NLCD_path, unzipped_NLCD_dir)

# Download and extract the elevation dataset
DEM_path = os.path.join(base_dir, 'elevation.zip')
download_and_extract(DEM_url, DEM_path, unzipped_DEM_dir)

# Download and extract the counties dataset
county_path = os.path.join(base_dir, 'counties.zip')
download_and_extract(county_url, county_path, unzipped_county_dir)

print("All folders successfully downloaded and extracted")

All folders successfully downloaded and extracted


In [11]:
# Define the new folder name
merged_fh = 'merged_data'

# Create a directory for the merged data
merged_folder = os.path.join(base_dir, merged_fh)
os.makedirs(merged_folder, exist_ok=True)

# List the subdirectories to merge
subdirectories = ['Unzipped Landcover', 'Unzipped Elevation', 'Unzipped Counties']

# Iterate through the subdirectories and copy their contents to the merged folder
for subdirectory in subdirectories:
    subdirectory_path = os.path.join(base_dir, subdirectory)
    if os.path.exists(subdirectory_path):
        for root, dirs, files in os.walk(subdirectory_path):
            for file in files:
                file_path = os.path.join(root, file)
                shutil.copy(file_path, merged_folder)
    print("Data from", subdirectory, "was merged")

Data from Unzipped Landcover was merged
Data from Unzipped Elevation was merged
Data from Unzipped Counties was merged


### File Setup Complete!

Now, our input data are conveniently merged into a single folder called 'merged_data'

**Action Required:** Move the 'mn_county_boundaries' shape files from merged_data dir to the parent directory, GIS_5571_Project.

In [36]:
import arcpy

# Set the workspace and input feature class
arcpy.env.workspace = r"C:\Users\jake1\OneDrive\Documents\ArcGIS\Projects\GIS_5571_Project"
input_feature_class = "mn_county_boundaries"

# Select county boundaries
selected = arcpy.management.SelectLayerByAttribute(input_feature_class, "NEW_SELECTION", where_clause = "CTY_Name IN ('Hennepin')")

# Create a feature layer with the selection
arcpy.management.CopyFeatures(selected, "Selected_Counties")

# Specify the output feature class for the selected features
output_feature_class = r"C:\Users\jake1\OneDrive\Documents\ArcGIS\Projects\GIS_5571_Project\mn_county_boundaries_clip"

print("Successfully clipped Hennepin County.")

Successfully clipped Hennepin County.


### ArcGIS Pro Map 

You should now see that two feature layers just appeared in your Contents Pane: 'Selected_Counties' and 'mn_county_boundaries_Layer2'.

Remove 'mn_county_boundaries_Layer2' so you can see only the Hennepin County boundaries.

Now that we have our boundaries for Hennepin County, we need to use the Extract by Mask function to extract our land cover data to *just* our Hennepin County polygon.

**Action Required:** Move the "NLCD_2019_Land_Cover.tif" files to the parent directory

In [2]:
# Confine land cover data to selected counties using extract by mask function
land_cover = arcpy.sa.ExtractByMask(
    in_raster = 'NLCD_2019_Land_Cover.tif',
    in_mask_data = 'Selected_Counties',
    extraction_area = 'INSIDE',
    analysis_extent = 'NLCD_2019_Land_Cover.tif'
)
print('Land Cover layer successfully extracted')

Land Cover layer successfully extracted


### Map Update

If correct, you should see the NLCD 2019 land cover data for Hennepin County in your contents pane as a raster layer named 'land_cover'. To see this on your map, unselect the 'Selected_Counties' layer.

Now, we need to add our DEM to the map. 

In [37]:
# Set the directory of the DEM .gdb to the workspace
arcpy.env.workspace = r'C:\Users\jake1\OneDrive\Documents\ArcGIS\Projects\GIS_5571_Project\Unzipped Elevation\elev_30m_digital_elevation_model.gdb'
# Name of DEM file: 'digital_elevation_model_30m'

# Use extract by mask function to extract the DEM inside Hennepin County 
DEM = arcpy.sa.ExtractByMask(
    in_raster = 'digital_elevation_model_30m',
    in_mask_data = 'Selected_Counties',
    extraction_area = 'INSIDE',
    analysis_extent = 'digital_elevation_model_30m'
)

print('DEM successfully extracted')

DEM successfully extracted


### Map Update

If correct, you should see the DEM data for Hennepin County in your contents pane as a geodatabase raster named 'DEM'.

Now, we need to adjust the slope of our DEM.

In [38]:
# Create string variable for .gdb directory
gdb_dir = os.path.join(base_dir, 'GIS_5571_Project.gdb')

# Set workspace to our .gdb directory
arcpy.env.workspace = gdb_dir

# Find slope from DEM
slope = arcpy.sa.Slope(
    in_raster = "DEM",
    output_measurement = "DEGREE",
    z_factor = 1,
    method = "PLANAR",
    z_unit = "METER")

print('Slope successfully extracted')

Slope successfully extracted


In [27]:
# Function to figure out slope reclassification values
def create_eql_classes(length, max_value):
    if length <= 0:
        return []  # Return an empty list if the length is non-positive

    # Calculate the value to be assigned to each element
    value = max_value / length

    # Create the list using a list comprehension
    result_list = [value * (i + 1) for i in range(length)]
    
    return result_list

# Initialize parameters
length = 5
max_value = 67.5146
# Call function
my_list = create_eql_classes(length, max_value)

# List output values
for i in my_list:
    print('value: {:.5f}'.format(i))

value: 13.50292
value: 27.00584
value: 40.50876
value: 54.01168
value: 67.51460


In [39]:
# Reclassify DEM slope
slope_reclass = arcpy.sa.Reclassify(
    in_raster = "slope",
    reclass_field = "VALUE",
    remap = "0 13.50292 5;13.50292 27.00584 4;27.00584 40.50876 3;40.50876 54.01168 2;54.01168 67.5146 1",
    missing_values = "DATA"
)

print('Slope reclassified')

Slope reclassified


In [3]:
# Reclassify NLCD data
reclass_NLCD = arcpy.sa.Reclassify(
    in_raster = "land_cover",
    reclass_field = "NLCD_Land",
    remap = "Developed, Open Space' 5;'Developed, Low Intensity' 3;'Developed, Medium Intensity' 3;'Developed, High Intensity' 3;'Barren Land' 5;'Deciduous Forest' 1;'Evergreen Forest' 1;'Mixed Forest' 1;Shrub/Scrub 5;Herbaceous 2;Hay/Pasture 4;'Cultivated Crops' 3",
    missing_values = "NODATA" # Open water and wetlands will turn to no data since it isn't remapped
)

print('Land cover reclassified')

Land cover reclassified


In [9]:
# Set the workspace and environment settings
arcpy.env.workspace = gdb_dir
arcpy.env.overwriteOutput = True

# Define the input data (Land Cover and DEM)
land_cover = "reclass_NLCD"
dem = "slope_reclass"

# Define the output folder for saving results
output_folder = base_dir

# Create a list of weight values to iterate through
weight_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# Iterate through different weight values
for weight in weight_values:
    # Calculate suitability using weighted overlay
    weighted_suitability = (arcpy.Raster(land_cover) * weight) + (arcpy.Raster(dem) * (1 - weight))

    # Save the suitability raster with a filename based on the weight
    output_name = f"Suitability_w{int(weight * 100)}.tif"
    output_path = os.path.join(output_folder, output_name)
    weighted_suitability.save(output_path)

print("Sensitivity analysis completed.")

Sensitivity analysis completed.


In [43]:
# Analysis of Suitability 

# Set your workspace folder where output files are located
arcpy.env.workspace = base_dir
workspace = base_dir

# List of output raster files
output_files = [os.path.join(workspace, filename) for filename in os.listdir(workspace) if filename.endswith('.tif')]

# Analyze each output file
for file_path in output_files:
    # Example analysis tasks (you can customize this)
    
    # Open the raster file
    raster = arcpy.Raster(file_path)  # Use arcpy or other libraries for different GIS formats
    
    # Calculate statistics
    mean = arcpy.GetRasterProperties_management(raster, "MEAN")
    min_value = arcpy.GetRasterProperties_management(raster, "MINIMUM")
    max_value = arcpy.GetRasterProperties_management(raster, "MAXIMUM")
    
    # Print or save the results
    print(f"Analysis for {file_path}:")
    print(f"Mean: {mean}")
    print(f"Minimum: {min_value}")
    print(f"Maximum: {max_value}")

Analysis for C:\Users\jake1\OneDrive\Documents\ArcGIS\Projects\GIS_5571_Project\NLCD_2019_Land_Cover.tif:
Mean: 60.410397229585
Minimum: 0
Maximum: 95
Analysis for C:\Users\jake1\OneDrive\Documents\ArcGIS\Projects\GIS_5571_Project\Suitability_w10.tif:
Mean: 4.7009858315724
Minimum: 1
Maximum: 5
Analysis for C:\Users\jake1\OneDrive\Documents\ArcGIS\Projects\GIS_5571_Project\Suitability_w20.tif:
Mean: 4.5378152426828
Minimum: 1
Maximum: 5
Analysis for C:\Users\jake1\OneDrive\Documents\ArcGIS\Projects\GIS_5571_Project\Suitability_w30.tif:
Mean: 4.3746448698364
Minimum: 1
Maximum: 5
Analysis for C:\Users\jake1\OneDrive\Documents\ArcGIS\Projects\GIS_5571_Project\Suitability_w40.tif:
Mean: 4.2114742820729
Minimum: 1
Maximum: 5
Analysis for C:\Users\jake1\OneDrive\Documents\ArcGIS\Projects\GIS_5571_Project\Suitability_w50.tif:
Mean: 4.04830393009
Minimum: 1
Maximum: 5
Analysis for C:\Users\jake1\OneDrive\Documents\ArcGIS\Projects\GIS_5571_Project\Suitability_w60.tif:
Mean: 3.8851335961208
Min

In [44]:
# List of output raster files
output_files = [os.path.join(workspace, filename) for filename in os.listdir(workspace) if filename.endswith('.tif')]

# Define the criteria names
criteria = ["Land Cover", "Slope"]

# Create an empty DataFrame to store the statistics
statistics_df = pd.DataFrame(columns=["File Name", "Criteria and Weight", "Mean", "Minimum", "Maximum"])

# Calculate the number of output files (excluding the first one)
num_files = len(output_files) - 1

# Analyze each output file (ignoring the first) and append statistics to the DataFrame
for i, file_path in enumerate(output_files[1:], start=1):
    # Example analysis tasks (you can customize this)
    
    # Open the raster file
    raster = arcpy.Raster(file_path)  # Use arcpy or other libraries for different GIS formats
    
    # Calculate statistics
    mean = arcpy.GetRasterProperties_management(raster, "MEAN")
    min_value = arcpy.GetRasterProperties_management(raster, "MINIMUM")
    max_value = arcpy.GetRasterProperties_management(raster, "MAXIMUM")
    
    # Extract the file name from the path
    file_name = os.path.basename(file_path)
    
    # Calculate weights based on linear progression, with 0.5 for the middle value
    weight_land_cover = i / num_files if i <= num_files / 2 else 1 - (i - num_files / 2) / num_files
    weight_slope = 1 - weight_land_cover
    
    # Create a single string for Criteria and Weight
    criteria_and_weight = f"{criteria[0]} Weight {weight_land_cover:.1f}, {criteria[1]} Weight {weight_slope:.1f}"
    
    # Append statistics to the DataFrame
    statistics_df = statistics_df.append({"File Name": file_name, "Criteria and Weight": criteria_and_weight, "Mean": mean, "Minimum": min_value, "Maximum": max_value}, ignore_index=True)

# Save the table to a CSV file
output_table = r"C:\Users\jake1\OneDrive\Documents\ArcGIS\Projects\GIS_5571_Project\results_table2.csv"
statistics_df.to_csv(output_table, index=False)

print('Table successfully created')

Table successfully created


In [10]:
# Add suitability scores to mpls solar installation points

# Set the workspace and raster layer
arcpy.env.workspace = gdb_dir 

# Set the paths to the feature classes and raster
point_feature_class = 'mpls_solar' # On-campus solar  
raster_path = 'Suitability_w50.tif'  

# Set the output shapefile name
output_shapefile = 'mpls_suitability.shp'

# Use Extract Values to Points tool
arcpy.sa.ExtractValuesToPoints(point_feature_class, raster_path, output_shapefile)

print(f"Extracted values saved to {output_shapefile}")

ExecuteError: ERROR 000622: Failed to execute (Extract Values to Points). Parameters are not valid.
ERROR 000628: Cannot set input into parameter in_raster.


In [18]:
# Create Pandas DF of Shapefile with Proj Suitability of UMN installations 

# Set the path to the UMN installations shp
extracted_shapefile = 'mpls_suitability.shp'  

# Read the attribute table into a Pandas DataFrame
fields = ['SHAPE@XY', 'Location', 'Size__kW_', 'RASTERVALU']  
data = []

with arcpy.da.SearchCursor(extracted_shapefile, fields) as cursor:
    for row in cursor:
        point = row[0]
        location = row[1]
        size = row[2]
        raster_value = row[3]

        data.append({
            'X': point[0],
            'Y': point[1],
            'Location': location,
            'Size__kW_': size,
            'RASTERVALU': raster_value
        })

# Create a Pandas DataFrame
df = pd.DataFrame(data)

# Print or use the DataFrame
print(df)

           X          Y                         Location Size__kW_  RASTERVALU
0 -93.231926  44.981432  Center for Excellence (Bierman)        98         4.0
1 -93.244488  44.973975                          Lot C86       537         4.0
2 -93.244297  44.973199                     Mondale Hall       184         4.0
3 -93.233207  44.976169                      Rapson Hall        45         4.0
4 -93.224031  44.974974          University Office Plaza        38         4.0
5 -93.243124  44.973861                  West Bank Field       311         3.5
6 -93.243548  44.973008                       Wiley Hall       109         4.0


In [28]:
# Adding True scores to the df

# List of scores from UMN Solar Tool
umn_scores = [4.28, 4.16, 4.32, 4.28, 4.2, 4.08, 4.2]
df['True_Score'] = umn_scores

# Calculate accuracy of est score vs UMN Solar Tool score
acc = df['RASTERVALU'] / df['True_Score']
df['Accuracy'] = acc

KeyError: 'RASTERVALU'

In [8]:
# Improve DF column names
df = df.rename(columns = {'RASTERVALU' : 'Est_Score', 'Size__kW_' : 'Size'})

NameError: name 'df' is not defined

### Validating scores against random points

In [7]:
base_dir = r"C:\Users\jake1\OneDrive\Documents\ArcGIS\Projects\GIS_5571_Project"
gdb_dir = os.path.join(base_dir, 'GIS_5571_Project.gdb')

# Set the workspace environment
arcpy.env.workspace = gdb_dir

In [33]:
# Set a seed for reproducibility
random_seed = 5571
random.seed(random_seed)

# Set the raster layer path
raster_path = "Selected_Counties"

# Get raster properties
desc = arcpy.Describe(raster_path)
extent = desc.extent

# Random points to generate
num_points = 26

# Create a list to store the random points
random_points = []

# Generate random points within the extent of the raster
for _ in range(num_points):
    # Ensure that the random point falls within the raster extent
    while True:
        random_x = random.uniform(extent.XMin, extent.XMax)
        random_y = random.uniform(extent.YMin, extent.YMax)
        if extent.XMin <= random_x <= extent.XMax and extent.YMin <= random_y <= extent.YMax:
            break
    random_points.append((random_x, random_y))

In [34]:
# Create a new point feature class
output_shapefile = "rand_pts"
arcpy.CreateFeatureclass_management(gdb_dir, "rand_pts", "POINT")

# Add a points to the feature class
with arcpy.da.InsertCursor(output_shapefile, ["SHAPE@XY"]) as cursor:
    for point in random_points:
        cursor.insertRow([point])

print(f"Shapefile created at {output_shapefile}")

Shapefile created at rand_pts


In [35]:
# Add suitability scores to mpls solar installation points

# Set workspace
arcpy.env.workspace = gdb_dir 

# Set shp and raster paths
point_feature_class = 'rand_pts'
raster_path = 'Suitability_w50.tif'  

# Set the output shapefile name
output_shapefile = 'rand_suitability.shp'

# Use Extract Values to Points tool
arcpy.sa.ExtractValuesToPoints(point_feature_class, raster_path, output_shapefile)

print(f"Extracted values saved to {output_shapefile}")

Extracted values saved to rand_suitability.shp


In [38]:
# Define WGS 84 CRS by describing mpls solar installations shp

dsc = arcpy.Describe('mpls_solar.shp')
coord_sys = dsc.spatialReference

# Set CRS to shapefile
arcpy.DefineProjection_management(output_shapefile, coord_sys)

print(f"The coordinate system of {point_feature_class} is now WGS 1984.")

The coordinate system of rand_pts is now WGS 1984.


In [39]:
# Add x and y coordinates to the attribute table (WGS 84)
shp_file = 'rand_suitability.shp'

arcpy.management.AddXY(shp_file)
print(f"Lat Lon coordinates now added to {shp_file} attribute table.")

Lat Lon coordinates now added to rand_suitability.shp attribute table.


In [41]:
# Create Pandas DF of Shapefile with random pts Suitability 

# Set the path to the UMN installations shp
extracted_shapefile = 'rand_suitability.shp'  

# Read the attribute table into a Pandas DataFrame
fields = ['SHAPE@XY', 'RASTERVALU']  
data = []

with arcpy.da.SearchCursor(extracted_shapefile, fields) as cursor:
    for row in cursor:
        point = row[0]
        raster_value = row[1]

        data.append({
            'X': point[0],
            'Y': point[1],
            'RASTERVALU': raster_value
        })

# Create a Pandas DataFrame
rand_df = pd.DataFrame(data)

no_data = -9999
rand_df = rand_df[rand_df['RASTERVALU'] != no_data]

In [42]:
# Output DataFrame to CSV file
csv_file_name = 'rand_suitability.csv'
csv_file_path = os.path.join(base_dir, csv_file_name)
rand_df.to_csv(csv_file_path, index = False)

print(f'DataFrame has been saved to {csv_file_path}')

DataFrame has been saved to C:\Users\jake1\OneDrive\Documents\ArcGIS\Projects\GIS_5571_Project\rand_suitability.csv


make sure reads as continuous scale
use color brewer